## Reproduce results from “Modeling BBR’s Interactions with Loss-Based Congestion Control”

This notebook will reproduce selected findings from

> Ranysha Ware, Matthew K. Mukerjee, Srinivasan Seshan, and Justine Sherry. 2019. Modeling BBR’s Interactions with Loss-Based Congestion Control. In Proceedings of the Internet Measurement Conference (IMC ’19). Association for Computing Machinery, New York, NY, USA, 137–143. https://doi.org/10.1145/3355369.3355604

specifically,

-   Figure 1b and 1c (initial measurements of BBR’s empirical behaviors)
-   Figure 2: BBR vs Cubic in a 40ms × 10Mbps network (convergence time and goodput for 1 BBR and 1 Cubic flow over varying queue sizes)
-   Figure 3: BBR and Cubic or Reno’s queue when competing for 4 minutes over a network with a 64 BDP (1024 packet) queue

The following group of cells installs prerequisites for data analysis and visualization:

In [ ]:
!pip install seaborn

In [ ]:
import time
import pandas as pd
from io import StringIO
import seaborn as sns 
import matplotlib.pyplot as plt

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="ware-" + fablib.get_bastion_username()

node_conf = [
 {'name': "sender",    'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['iperf3']}, 
 {'name': "receiver",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': ['iperf3']}, 
 {'name': "router",    'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}
]
net_conf = [
 {"name": "net0", "subnet": "10.0.0.0/24", "nodes": [{"name": "sender",   "addr": "10.0.0.100"}, {"name": "router", "addr": "10.0.0.1"}]},
 {"name": "net1", "subnet": "10.0.1.0/24", "nodes": [{"name": "receiver", "addr": "10.0.1.100"}, {"name": "router", "addr": "10.0.1.1"}]},
]
route_conf = [
 {"addr": "10.0.1.0/24", "gw": "10.0.0.1", "nodes": ["sender"]}, 
 {"addr": "10.0.0.0/24", "gw": "10.0.1.1", "nodes": ["receiver"]}
]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo DEBIAN_FRONTEND=noninteractive apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# make sure all interfaces are brought up
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Validate base network

Before we run any experiment, we should check the “base” network - before adding any emulated delay or rate limiting - and make sure that it will not be a limiting factor in the experiment.

In [ ]:
# check base delay
_ = slice.get_node("sender").execute("ping -c 5 receiver")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
_ = slice.get_node("receiver").execute("iperf3 -s -1 -D")
time.sleep(5)
_ = slice.get_node("sender").execute("iperf3 -t 30 -i 10 -P 10 -c receiver")

In [ ]:
# also check Linux kernel version on sender
_ = slice.get_node("sender").execute("uname -a")

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Configure the network capacity and delay

In this section, we configure the bottleneck link to have a 40Mbps capacity and 10ms delay. We will initialize the queue size to 32 BDP (although we will change this later).

In [ ]:
exp = {'rtt': 10, 'bandwidth': 40 }
bdp_kbyte = exp['rtt']*exp['bandwidth']/8

router_node = slice.get_node("router")
router_ingress_iface = router_node.get_interface(network_name = "net0")
router_ingress_name = router_ingress_iface.get_device_name()
router_egress_iface  = router_node.get_interface(network_name = "net1")
router_egress_name = router_egress_iface.get_device_name()

router_node.execute("sudo tc qdisc del dev " + router_ingress_name + " root")
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

# set up RTT
router_node.execute("sudo tc qdisc replace dev " + router_ingress_name + " root netem delay " + str(exp['rtt']) + "ms limit 10000")
# set up rate limit, buffer limit
router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3")
router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate " + str(exp['bandwidth']) + "Mbit")
router_node.execute("sudo tc qdisc add dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*32) + "kb")


Then, we validate the new network setting.

In [ ]:
# check base delay
_ = slice.get_node("sender").execute("ping -c 5 receiver")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
_ = slice.get_node("receiver").execute("iperf3 -s -1 -D")
time.sleep(5)
_ = slice.get_node("sender").execute("iperf3 -t 30 -i 10 -P 10 -c receiver")

### Reproduce Figure 1b

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
import itertools
exp_factors = { 
    'bufcap': [0.25, 0.5] + [2**n for n in range(8)],
    'duration': [240],
    'loss_cc': ['cubic', 'reno'],
    'trial': [1]
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]

In [ ]:
import time

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")

for exp in exp_lists:
    # set router buffer limit 
    router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*exp['bufcap']) + "kb")

    # clean up
    receiver_node.execute("sudo killall iperf3")
    sender_node.execute("sudo killall iperf3")

    # start an iperf3 receiver for the BBR flow
    receiver_node.execute_thread("iperf3 -s -1 -i 1 --logfile fig1b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt")
    # start an iperf3 receiver for the Cubic flows
    receiver_node.execute_thread("iperf3 -s -1 -i 1 --logfile fig1b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt -p 5301")

    time.sleep(5) 

    # start an iperf3 sender for the BBR flow
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C bbr ")
    # start an iperf3 receiver for the Cubic flows
    sender_node.execute_thread("iperf3 -c receiver -fm -t " + str(exp['duration']) + " -C " + exp['loss_cc'] + "-p 5301")

    time.sleep(exp['duration'] + 5)


In [ ]:
df = pd.DataFrame(columns=['bufcap', 'combo', 'cc', 'goodput'])

for exp in exp_lists:

    bbr_file = "fig1b_bbr_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_bbr = receiver_node.execute("cat " + bbr_file + " | grep 'receiver' | awk -F '-' '{print $2}' | awk '{print $5}'", quiet=True)
    df_dict = {'bufcap': exp['bufcap'], 'combo': "BBR-" + exp['loss_cc'], 'cc': 'BBR', 'goodput': float(tput_bbr[0].strip())}
    df = pd.concat([df, pd.DataFrame(df_dict, index=[0])], ignore_index=True)


    loss_file = "fig1b_" + exp['loss_cc'] + "_" + str(exp['bufcap']) + "_bbrV" + exp['loss_cc'] + ".txt"
    tput_loss = receiver_node.execute("cat " + cubic_file + " | grep 'receiver' | awk -F '-' '{print $2}' | awk '{print $5}'", quiet=True)
    df_dict = {'bufcap': exp['bufcap'], 'combo': "BBR-" + exp['loss_cc'], 'cc': exp['loss_cc'], 'goodput': float(tput_loss[0].strip())}
    df = pd.concat([df, pd.DataFrame(df_dict, index=[0])], ignore_index=True)


### Reproduce Figure 1c

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
# set router buffer limit to 32 BDP
router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bdp_kbyte*32) + "kb")

# make sure BBR is available
sender_node.execute("sudo modprobe tcp_bbr")

# clean up
receiver_node.execute("sudo killall iperf3")
receiver_node.execute("rm fig1c_bbr.txt")
receiver_node.execute("rm fig1c_cubic.txt")

# start an iperf3 receiver for the BBR flow
receiver_node.execute_thread("iperf3 -s -1 -i 1 --logfile fig1c_bbr.txt")
# start an iperf3 receiver for the Cubic flows
receiver_node.execute_thread("iperf3 -s -1 -i 1 --logfile fig1c_cubic.txt -p 5301")

time.sleep(5) 

# start an iperf3 sender for the BBR flow
sender_node.execute_thread("iperf3 -c receiver -fm -t 300 -C bbr ")
# start an iperf3 receiver for the Cubic flows
sender_node.execute_thread("iperf3 -c receiver -fm -t 300 -C cubic -P 16 -p 5301")

time.sleep(305)

In [ ]:
tput_bbr = receiver_node.execute("head --lines=-5 fig1c_bbr.txt | grep 'Mbits/sec' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
df_bbr = pd.read_csv(StringIO(tput_bbr[0]), names = ['time','goodput'])

tput_cubic = receiver_node.execute("head --lines=-37 fig1c_cubic.txt | grep 'SUM' | awk -F '-' '{print $2}' | awk '{print $1\",\"$5}'", quiet=True)
df_cubic = pd.read_csv(StringIO(tput_cubic[0]), names = ['time','goodput'])

_ = plt.figure(figsize=(6,3))
_ = plt.plot(df_bbr.time, df_bbr.goodput, label="1 BBR flow")
_ = plt.plot(df_cubic.time, df_cubic.goodput, label="Sum of 16 CUBIC flows")
_ = plt.legend(loc="upper center", ncol=2)
_ = plt.ylabel("Goodput (Mbps)")
_ = plt.xlabel("Time (s)")

### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()